# StatsExchange - Predicting users' reputations

In this notebook, we will use relational learning techniques to predict users' reputation on StackExchange

Summary:

- Prediction type: __Regression model__
- Domain: __Internet__
- Prediction target: __Reputation__ 
- Population size: __41793__

## Background

StatsExchange is a website similar to StackOverflow, but based on statistics and machine learning. Much like StackOverflow, it has a complicated system of calculating users' reputation. In this notebook, we will benchmark relational learning techniques based on there ability to reverse-engineer said system.

The dataset has been downloaded from the [CTU Prague relational learning repository](https://relational.fit.cvut.cz/dataset/Stats) (Motl and Schulte, 2015)(Now residing at [relational-data.org](https://relational-data.org/dataset/Stats).).

We will benchmark [getML](https://www.getml.com) 's feature learning algorithms against [featuretools](https://www.featuretools.com), an open-source implementation of the propositionalization algorithm, similar to getML's FastProp.

## Analysis

Let's get started with the analysis and set up your session:

In [1]:
%pip install -q "getml==1.5.0" "featuretools==1.31.0" "ipywidgets==8.1.5"

In [1]:
import os
import warnings

import pandas as pd

import featuretools
import woodwork as ww
import getml

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
warnings.simplefilter(action='ignore', category=FutureWarning)

print(f"getML API version: {getml.__version__}\n")

getML API version: 1.5.0



In [2]:
getml.engine.launch(allow_remote_ips=True, token='token')
getml.engine.set_project('stats')

Launching ./getML --allow-push-notifications=true --allow-remote-ips=true --home-directory=/home/user --in-memory=true --install=false --launch-browser=true --log=false --token=token in /home/user/.getML/getml-1.5.0-x64-linux...
Launched the getML Engine. The log output will be stored in /home/user/.getML/logs/20240912151014.log.
  Loading pipelines... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


Connected to project 'stats'.

### 1. Loading data

#### 1.1 Download from source

We begin by downloading the data:

In [4]:
conn = getml.database.connect_mysql(
    host="db.relational-data.org",
    dbname="stats",
    port=3306,
    user="guest",
    password="relational"
)

conn

Connection(dbname='stats', dialect='mysql', host='db.relational-data.org', port=3306)

In [5]:
def load_if_needed(name):
    """
    Loads the data from the relational learning
    repository, if the data frame has not already
    been loaded.
    """
    if not getml.data.exists(name):
        data_frame = getml.data.DataFrame.from_db(
            name=name,
            table_name=name,
            conn=conn
        )
        data_frame.save()
    else:
        data_frame = getml.data.load_data_frame(name)
    return data_frame

In [6]:
badges = load_if_needed("badges")
posts = load_if_needed("posts")
users = load_if_needed("users")
votes = load_if_needed("votes")

In [7]:
badges

name             Id         UserId   Name             Date               
 role   unused_float   unused_float   unused_string    unused_string      
    0              1              5   Teacher          2010-07-19 19:39:07
    1              2              6   Teacher          2010-07-19 19:39:07
    2              3              8   Teacher          2010-07-19 19:39:07
    3              4             23   Teacher          2010-07-19 19:39:07
    4              5             36   Teacher          2010-07-19 19:39:07
                 ...            ...   ...              ...                
79846          92236          55744   Student          2014-09-13 23:25:21
79847          92237           1118   Nice Answer      2014-09-14 00:09:35
79848          92238           1118   Enlightened      2014-09-14 01:18:29
79849          92239          55746   Student          2014-09-14 01:41:18
79850          92240          12597   Autobiographer   2014-09-14 02:31:28


79851 rows x 4 columns
memory usage: 4.96 MB
type: getml.DataFrame

In [8]:
posts

name             Id     PostTypeId   AcceptedAnswerId          Score   ...   LastEditDate          CommunityOwnedDate 
 role   unused_float   unused_float       unused_float   unused_float   ...   unused_string         unused_string      
    0              1              1                15              23   ...   NULL                  NULL               
    1              2              1                59              22   ...   2010-08-07 17:56:44   NULL               
    2              3              1                 5              54   ...   2011-02-12 05:50:03   2010-07-19 19:13:28
    3              4              1               135              13   ...   NULL                  NULL               
    4              5              2               nan              81   ...   2010-07-19 19:21:15   2010-07-19 19:14:43
                 ...            ...                ...            ...         ...                   ...                
91971         115374              2               nan               2   ...   2014-09-14 02:05:41   NULL               
91972         115375              1               nan               0   ...   NULL                  NULL               
91973         115376              1               nan               1   ...   2014-09-14 01:40:55   NULL               
91974         115377              2               nan               0   ...   2014-09-14 02:54:13   NULL               
91975         115378              2               nan               0   ...   NULL                  NULL               

 name   ClosedDate      OwnerDisplayName   LastEditorDisplayName
 role   unused_string   unused_string      unused_string        
    0   NULL            NULL               NULL                 
    1   NULL            NULL               NULL                 
    2   NULL            NULL               NULL                 
    3   NULL            NULL               NULL                 
    4   NULL            NULL               NULL                 
        ...             ...                ...                  
91971   NULL            NULL               NULL                 
91972   NULL            NULL               NULL                 
91973   NULL            NULL               NULL                 
91974   NULL            NULL               NULL                 
91975   NULL            NULL               NULL                 


91976 rows x 21 columns
memory usage: 128.41 MB
type: getml.DataFrame

In [9]:
users

name             Id     Reputation          Views        UpVotes   ...   LastAccessDate        WebsiteUrl                    
 role   unused_float   unused_float   unused_float   unused_float   ...   unused_string         unused_string                 
    0            -1               1              0           5007   ...   2010-07-19 06:55:26   http://meta.stackexchange.com/
    1             2             101             25              3   ...   2013-11-12 22:07:23   http://stackoverflow.com      
    2             3             101             22             19   ...   2014-08-08 06:42:58   http://stackoverflow.com      
    3             4             101             11              0   ...   2014-01-02 09:31:02   http://minesweeperonline.com  
    4             5            6792           1145            662   ...   2014-08-13 00:23:47   http://www.statalgo.com       
                 ...            ...            ...            ...         ...                   ...                           
40320         55743               1              0              0   ...   2014-09-13 21:18:52   NULL                          
40321         55744               6              1              0   ...   2014-09-13 21:39:30   NULL                          
40322         55745             101              0              0   ...   2014-09-13 23:45:27   NULL                          
40323         55746             106              1              0   ...   2014-09-14 02:05:17   NULL                          
40324         55747               1              0              0   ...   2014-09-14 01:19:04   NULL                          

 name   Location             AboutMe                            ProfileImageUrl                 
 role   unused_string        unused_string                      unused_string                   
    0   on the server farm   <p>Hi, I'm not really a person.<...   NULL                            
    1   Corvallis, OR        <p>Developer on the StackOverflo...   NULL                            
    2   New York, NY         <p><a href="http://blog.stackove...   NULL                            
    3   San Francisco, CA    <p>currently at a startup in SF<...   http://i.stack.imgur.com/d1oHX.j...
    4   New York, NY         <p>Quantitative researcher focus...   NULL                            
        ...                  ...                                ...                             
40320   NULL                 NULL                               http://graph.facebook.com/665821...
40321   NULL                 NULL                               NULL                            
40322   United States        NULL                               https://www.gravatar.com/avatar/...
40323   NULL                 <p>Stats noobie, product, market...   https://www.gravatar.com/avatar/...
40324   India                <p>Maths Enthusiast </p>
          https://lh4.googleusercontent.co...


40325 rows x 14 columns
memory usage: 10.32 MB
type: getml.DataFrame

In [10]:
votes

name             Id         PostId     VoteTypeId         UserId   BountyAmount   CreationDate 
  role   unused_float   unused_float   unused_float   unused_float   unused_float   unused_string
     0              1              3              2           nan            nan    2010-07-19   
     1              2              2              2           nan            nan    2010-07-19   
     2              3              5              2           nan            nan    2010-07-19   
     3              4              5              2           nan            nan    2010-07-19   
     4              5              3              2           nan            nan    2010-07-19   
                  ...            ...            ...            ...            ...   ...          
328059         386254          26088              2           nan            nan    2014-09-14   
328060         386255          26088              5         31466            nan    2014-09-14   
328061         386256         115374              2           nan            nan    2014-09-14   
328062         386257         115368              2           nan            nan    2014-09-14   
328063         386258         115369              2           nan            nan    2014-09-14   


328064 rows x 6 columns
memory usage: 19.36 MB
type: getml.DataFrame

#### 1.2 Prepare data for getML

getML requires that we define *roles* for each of the columns.

In [11]:
badges.set_role(["Id", "UserId"], getml.data.roles.join_key)
badges.set_role("Date", getml.data.roles.time_stamp)
badges.set_role("Name", getml.data.roles.categorical)

badges

name                          Date         Id     UserId   Name          
 role                    time_stamp   join_key   join_key   categorical   
 unit   time stamp, comparison only                                       
    0           2010-07-19 19:39:07          1          5   Teacher       
    1           2010-07-19 19:39:07          2          6   Teacher       
    2           2010-07-19 19:39:07          3          8   Teacher       
    3           2010-07-19 19:39:07          4         23   Teacher       
    4           2010-07-19 19:39:07          5         36   Teacher       
                                ...        ...        ...   ...           
79846           2014-09-13 23:25:21      92236      55744   Student       
79847           2014-09-14 00:09:35      92237       1118   Nice Answer   
79848           2014-09-14 01:18:29      92238       1118   Enlightened   
79849           2014-09-14 01:41:18      92239      55746   Student       
79850           2014-09-14 02:31:28      92240      12597   Autobiographer


79851 rows x 4 columns
memory usage: 1.60 MB
type: getml.DataFrame

In [12]:
posts.set_role(["Id", "AcceptedAnswerId", "OwnerUserId"], getml.data.roles.join_key)
posts.set_role(["Score", "ViewCount", "AnswerCount", "CommentCount", "FavoriteCount", "LastEditorUserId"], getml.data.roles.numerical)
posts.set_role(["PostTypeId"], getml.data.roles.categorical)

posts.drop(posts.roles.unused)

name         Id   AcceptedAnswerId   OwnerUserId   PostTypeId    ...   ViewCount   AnswerCount   CommentCount   FavoriteCount
role   join_key           join_key      join_key   categorical   ...   numerical     numerical      numerical       numerical
   0          1                15              8   1             ...       1278             5               1             14 
   1          2                59             24   1             ...       8198             7               1              8 
   2          3                 5             18   1             ...       3613            19               4             36 
   3          4               135             23   1             ...       5224             5               2              2 
   4          5               nan             23   2             ...        nan           nan               3            nan 
 ...        ...                ...           ...   ...           ...         ...           ...            ...             ...

name   LastEditorUserId
role          numerical
   0               nan 
   1                88 
   2               183 
   3               nan 
   4                23 
 ...                ...


91976 rows x 10 columns
type: getml.data.View

In [13]:
users.set_role("Reputation", getml.data.roles.target)
users.set_role(["CreationDate", "LastAccessDate"], getml.data.roles.time_stamp)
users.set_role(["Id"], getml.data.roles.join_key)
users.set_role(["Views", "UpVotes", "DownVotes"], getml.data.roles.numerical)

users.drop(users.roles.unused)

name                  CreationDate                LastAccessDate         Id   Reputation       Views     UpVotes   DownVotes
role                    time_stamp                    time_stamp   join_key       target   numerical   numerical   numerical
unit   time stamp, comparison only   time stamp, comparison only                                                            
   0           2010-07-19 06:55:26           2010-07-19 06:55:26        -1             1           0        5007        1920
   1           2010-07-19 14:01:36           2013-11-12 22:07:23         2           101          25           3           0
   2           2010-07-19 15:34:50           2014-08-08 06:42:58         3           101          22          19           0
   3           2010-07-19 19:03:27           2014-01-02 09:31:02         4           101          11           0           0
   4           2010-07-19 19:03:57           2014-08-13 00:23:47         5          6792        1145         662           5
 ...                           ...                           ...        ...          ...         ...         ...         ...


40325 rows x 7 columns
type: getml.data.View

In [14]:
votes.set_role(["CreationDate"], getml.data.roles.time_stamp)
votes.set_role(["Id", "PostId"], getml.data.roles.join_key)
votes.set_role(["VoteTypeId"], getml.data.roles.categorical)
votes.set_role(["BountyAmount"], getml.data.roles.numerical)

votes.drop(users.roles.unused)

name                  CreationDate         Id     PostId   VoteTypeId    BountyAmount         UserId
role                    time_stamp   join_key   join_key   categorical      numerical   unused_float
unit   time stamp, comparison only                                                                  
   0                    2010-07-19          1          3   2                     nan            nan 
   1                    2010-07-19          2          2   2                     nan            nan 
   2                    2010-07-19          3          5   2                     nan            nan 
   3                    2010-07-19          4          5   2                     nan            nan 
   4                    2010-07-19          5          3   2                     nan            nan 
 ...                           ...        ...        ...   ...                    ...            ...


328064 rows x 6 columns
type: getml.data.View

In [15]:
split = getml.data.split.random(train=0.8, test=0.2)

In [16]:
container = getml.data.Container(population=users, split=split)

container.add(badges=badges, posts=posts, votes=votes)

container

population
    subset   name     rows   type
0   test     users    7992   View
1   train    users   32333   View

peripheral
    name       rows   type     
0   badges    79851   DataFrame
1   posts     91976   DataFrame
2   votes    328064   DataFrame

### 2. Predictive modeling

We loaded the data and defined the roles and units. Next, we create a getML pipeline for relational learning.

#### 2.1 Define relational model

In [17]:
dm = getml.data.DataModel(population=users.to_placeholder())

dm.add(getml.data.to_placeholder(badges=badges, posts=[posts]*2, votes=votes))

dm.population.join(
    dm.badges,
    on=("Id", "UserId"),
)

dm.population.join(
    dm.posts[0],
    on=("Id", "OwnerUserId"),
)

dm.posts[0].join(
    dm.posts[1],
    on=("Id", "AcceptedAnswerId"),
    relationship=getml.data.relationship.one_to_one,
)

dm.posts[0].join(
    dm.votes,
    on=("Id", "PostId"),
)

dm

users:
  columns:
  - Id: join_key
  - Views: numerical
  - UpVotes: numerical
  - DownVotes: numerical
  - Reputation: target
  - ...

  joins:
  - right: 'badges'
    on: (users.Id, badges.UserId)
    relationship: 'many-to-many'
    lagged_targets: False
  - right: 'posts'
    on: (users.Id, posts.OwnerUserId)
    relationship: 'many-to-many'
    lagged_targets: False

badges:
  columns:
  - Name: categorical
  - Id: join_key
  - UserId: join_key
  - Date: time_stamp

posts:
  columns:
  - PostTypeId: categorical
  - Id: join_key
  - AcceptedAnswerId: join_key
  - OwnerUserId: join_key
  - Score: numerical
  - ...

  joins:
  - right: 'posts'
    on: (posts.Id, posts.AcceptedAnswerId)
    relationship: 'one-to-one'
    lagged_targets: False
  - right: 'votes'
    on: (posts.Id, votes.PostId)
    relationship: 'many-to-many'
    lagged_targets: False

posts:
  columns:
  - PostTypeId: categorical
  - Id: join_key
  - AcceptedAnswerId: join_key
  - OwnerUserId: join_key
  - Score: numerical
  - ...

votes:
  columns:
  - VoteTypeId: categorical
  - Id: join_key
  - PostId: join_key
  - BountyAmount: numerical
  - CreationDate: time_stamp
  - ...

#### 2.2 getML pipeline

<!-- #### 2.1.1  -->
__Set-up the feature learner & predictor__

We use the relboost algorithms for this problem. Because of the large number of keywords, we regularize the model a bit by requiring a minimum support for the keywords (`min_num_samples`).

In [18]:
mapping = getml.preprocessors.Mapping()

fast_prop = getml.feature_learning.FastProp(
    aggregation=["AVG", "SUM"],
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,
    n_most_frequent=5,
    
)

relboost = getml.feature_learning.Relboost(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,
    num_subfeatures=10,
)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

__Build the pipeline__

In [19]:
pipe1 = getml.Pipeline(
    tags=['fast_prop'],
    data_model=dm,
    feature_learners=[fast_prop],
    predictors=[predictor],
    include_categorical=True,
)

pipe1

Pipeline(data_model='users',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=True,
         loss_function='SquareLoss',
         peripheral=['badges', 'posts', 'votes'],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['fast_prop'])

In [20]:
pipe2 = getml.Pipeline(
    tags=['relboost'],
    data_model=dm,
    preprocessors=[mapping],
    feature_learners=[relboost],
    predictors=[predictor],    
    include_categorical=True,
)

pipe2

Pipeline(data_model='users',
         feature_learners=['Relboost'],
         feature_selectors=[],
         include_categorical=True,
         loss_function='SquareLoss',
         peripheral=['badges', 'posts', 'votes'],
         predictors=['XGBoostRegressor'],
         preprocessors=['Mapping'],
         share_selected_features=0.5,
         tags=['relboost'])

#### 2.3 Model training

In [21]:
pipe1.check(container.train)

Checking data model...

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
⠋ Checking... ━━━━━━━━━━━━━━╸                           37% • 00:01

  Checking... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


The pipeline check generated 3 issues labeled INFO and 3 issues labeled WARNING.

type      label                     message                         
0   WARNING   COLUMN SHOULD BE UNUSED   All non-NULL entries in column '...
1   WARNING   COLUMN SHOULD BE UNUSED   93.147125% of all entries in col...
2   WARNING   COLUMN SHOULD BE UNUSED   99.468396% of all entries in col...
3   INFO      FOREIGN KEYS NOT FOUND    When joining USERS__STAGING_TABL...
4   INFO      FOREIGN KEYS NOT FOUND    When joining USERS__STAGING_TABL...
5   INFO      FOREIGN KEYS NOT FOUND    When joining POSTS__STAGING_TABL...

In [22]:
pipe1.fit(container.train)

Checking data model...

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


The pipeline check generated 3 issues labeled INFO and 3 issues labeled WARNING.

To see the issues in full, run .check() on the pipeline.

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  FastProp: Trying 522 features... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:08
  FastProp: Building subfeatures... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  FastProp: Building features... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:01
  XGBoost: Training as predictor... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:09


Trained pipeline.

Time taken: 0:00:19.835015.



Pipeline(data_model='users',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=True,
         loss_function='SquareLoss',
         peripheral=['badges', 'posts', 'votes'],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['fast_prop', 'container-1mZfCR'])

In [23]:
pipe2.check(container.train)

Checking data model...

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Checking... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


The pipeline check generated 3 issues labeled INFO and 3 issues labeled WARNING.

type      label                     message                         
0   WARNING   COLUMN SHOULD BE UNUSED   All non-NULL entries in column '...
1   WARNING   COLUMN SHOULD BE UNUSED   93.147125% of all entries in col...
2   WARNING   COLUMN SHOULD BE UNUSED   99.468396% of all entries in col...
3   INFO      FOREIGN KEYS NOT FOUND    When joining USERS__STAGING_TABL...
4   INFO      FOREIGN KEYS NOT FOUND    When joining USERS__STAGING_TABL...
5   INFO      FOREIGN KEYS NOT FOUND    When joining POSTS__STAGING_TABL...

In [24]:
pipe2.fit(container.train)

Checking data model...

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


The pipeline check generated 3 issues labeled INFO and 3 issues labeled WARNING.

To see the issues in full, run .check() on the pipeline.

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Relboost: Training subfeatures... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:19
  Relboost: Training subfeatures... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:22
  Relboost: Building subfeatures... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Relboost: Building subfeatures... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Relboost: Training features... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 01:10
  Relboost: Building subfeatures... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Relboost: Building subfeatures... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Relboost: Building features... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:04
  XGBoost: Training as predictor... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:04


Trained pipeline.

Time taken: 0:02:05.355604.



Pipeline(data_model='users',
         feature_learners=['Relboost'],
         feature_selectors=[],
         include_categorical=True,
         loss_function='SquareLoss',
         peripheral=['badges', 'posts', 'votes'],
         predictors=['XGBoostRegressor'],
         preprocessors=['Mapping'],
         share_selected_features=0.5,
         tags=['relboost', 'container-1mZfCR'])

#### 2.4 Model evaluation

In [25]:
fastprop_score = pipe1.score(container.test)
fastprop_score

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  FastProp: Building subfeatures... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  FastProp: Building features... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


date time             set used   target            mae       rmse   rsquared
0   2024-09-12 12:57:46   train      Reputation    31.8897    43.6771     0.9974
1   2024-09-12 12:59:53   test       Reputation    33.6064    65.3332     0.9777

In [26]:
relboost_score = pipe2.score(container.test)
relboost_score

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Relboost: Building subfeatures... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Relboost: Building subfeatures... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Relboost: Building features... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:01


date time             set used   target            mae       rmse   rsquared
0   2024-09-12 12:59:52   train      Reputation    29.7673    42.9683     0.9975
1   2024-09-12 12:59:55   test       Reputation    32.2748    75.6576     0.9728

#### 2.5 featuretools

In [27]:
population_train_pd = container.train.population.to_pandas()
population_test_pd = container.test.population.to_pandas()

In [28]:
badges_pd = badges.drop(badges.roles.unused).to_pandas()
posts_pd = posts.drop(posts.roles.unused).to_pandas()
votes_pd = votes.drop(votes.roles.unused).to_pandas()

In [29]:
population_pd_logical_types = {
    "Id": ww.logical_types.IntegerNullable,
    "Views": ww.logical_types.Integer,
    "UpVotes": ww.logical_types.Integer,
    "DownVotes": ww.logical_types.Integer,
    "Reputation": ww.logical_types.Integer,
    "CreationDate": ww.logical_types.Datetime,
    "LastAccessDate": ww.logical_types.Datetime
}
population_train_pd.ww.init(logical_types=population_pd_logical_types, index="Id", name="population")
population_test_pd.ww.init(logical_types=population_pd_logical_types, index="Id", name="population")

badges_pd_logical_types = {
    "Name": ww.logical_types.Categorical,
    "Id": ww.logical_types.Integer,
    "UserId": ww.logical_types.IntegerNullable,
    "Date": ww.logical_types.Datetime
}
badges_pd.ww.init(logical_types=badges_pd_logical_types, index="Id", name="badges")

posts_pd_logical_types = {
    "PostTypeId": ww.logical_types.Integer,
    "Id": ww.logical_types.Integer,
    "AcceptedAnswerId": ww.logical_types.IntegerNullable,
    "OwnerUserId": ww.logical_types.IntegerNullable,
    "Score": ww.logical_types.Integer,
    "ViewCount": ww.logical_types.IntegerNullable,
    "AnswerCount": ww.logical_types.IntegerNullable,
    "CommentCount": ww.logical_types.Integer,
    "FavoriteCount": ww.logical_types.IntegerNullable,
    "LastEditorUserId": ww.logical_types.IntegerNullable
}
posts_pd.ww.init(logical_types=posts_pd_logical_types, index="Id", name="posts")

votes_pd_logical_types = {
    "VoteTypeId": ww.logical_types.Categorical,
    "Id": ww.logical_types.Integer,
    "PostId": ww.logical_types.Integer,
    "BountyAmount": ww.logical_types.IntegerNullable,
    "CreationDate": ww.logical_types.Datetime
}
votes_pd.ww.init(logical_types=votes_pd_logical_types, index="Id", name="votes")

In [30]:
dataframes_train = {
    "population" : (population_train_pd, ),
    "badges": (badges_pd, ),
    "posts": (posts_pd, ),
    "votes": (votes_pd, ),
}

In [31]:
dataframes_test = {
    "population" : (population_test_pd, ),
    "badges": (badges_pd, ),
    "posts": (posts_pd, ),
    "votes": (votes_pd, ),
}

In [32]:
relationships = [
    ("population", "Id", "badges", "UserId"),
    ("population", "Id", "posts", "OwnerUserId"),
    ("posts", "Id", "votes", "PostId"),
]

In [33]:
featuretools_train_pd = featuretools.dfs(
    dataframes=dataframes_train,
    relationships=relationships,
    target_dataframe_name="population")[0]

In [34]:
featuretools_test_pd = featuretools.dfs(
    dataframes=dataframes_test,
    relationships=relationships,
    target_dataframe_name="population")[0]

In [35]:
featuretools_train = getml.data.DataFrame.from_pandas(featuretools_train_pd, "featuretools_train")
featuretools_test = getml.data.DataFrame.from_pandas(featuretools_test_pd, "featuretools_test")

In [36]:
featuretools_train.set_role("Reputation", getml.data.roles.target)
featuretools_train.set_role(featuretools_train.roles.unused_float, getml.data.roles.numerical)
featuretools_train.set_role(featuretools_train.roles.unused_string, getml.data.roles.categorical)
featuretools_train.set_role(["SKEW(posts.STD(votes.BountyAmount))", "STD(posts.SKEW(votes.BountyAmount))"], getml.data.roles.unused_float)

featuretools_train

name   Reputation   COUNT(badges)   MODE(badges.Name)   NUM_UNIQUE(badges.Name)   ...   SUM(votes.posts.PostTypeId)
 role       target   categorical     categorical         categorical               ...                     numerical
    0            1   0               NULL                NULL                      ...                            44
    1          101   3               Autobiographer      3                         ...                             0
    2          101   3               Autobiographer      3                         ...                             0
    3         6792   131             Nice Answer         45                        ...                          2953
    4          457   19              Announcer           16                        ...                           158
               ...   ...             ...                 ...                                                     ...
32328            1   0               NULL                NULL                      ...                             0
32329          101   1               Supporter           1                         ...                             0
32330            6   1               Student             1                         ...                             2
32331          101   0               NULL                NULL                      ...                             0
32332            1   0               NULL                NULL                      ...                             0

 name   SUM(votes.posts.Score)   SUM(votes.posts.ViewCount)   SKEW(posts.STD(votes.BountyAmount))
 role                numerical                    numerical                       unused_float
    0                        0                            0                               nan 
    1                        0                            0                               nan 
    2                        0                            0                               nan 
    3                   130132                     33354308                               nan 
    4                     1998                        21201                               nan 
                           ...                          ...                                ...
32328                        0                            0                               nan 
32329                        0                            0                               nan 
32330                        2                           34                               nan 
32331                        0                            0                               nan 
32332                        0                            0                               nan 

 name   STD(posts.SKEW(votes.BountyAmount))
 role                       unused_float
    0                               nan 
    1                               nan 
    2                               nan 
    3                               nan 
    4                               nan 
                                     ...
32328                               nan 
32329                               nan 
32330                               nan 
32331                               nan 
32332                               nan 


32333 rows x 173 columns
memory usage: 41.52 MB
type: getml.DataFrame

In [37]:
featuretools_test.set_role("Reputation", getml.data.roles.target)
featuretools_test.set_role(featuretools_test.roles.unused_float, getml.data.roles.numerical)
featuretools_test.set_role(featuretools_test.roles.unused_string, getml.data.roles.categorical)
featuretools_test.set_role(["SKEW(posts.STD(votes.BountyAmount))", "STD(posts.SKEW(votes.BountyAmount))"], getml.data.roles.unused_float)

featuretools_test

name   Reputation   COUNT(badges)   MODE(badges.Name)   NUM_UNIQUE(badges.Name)   ...   SUM(votes.posts.PostTypeId)
role       target   categorical     categorical         categorical               ...                     numerical
   0          101   2               Autobiographer      2                         ...                             0
   1          429   10              Yearling            9                         ...                            88
   2          101   5               Autobiographer      5                         ...                             4
   3         2185   44              Announcer           20                        ...                           756
   4         1651   22              Nice Answer         14                        ...                           315
              ...   ...             ...                 ...                                                     ...
7987            1   0               NULL                NULL                      ...                             0
7988           11   1               Student             1                         ...                             2
7989            1   0               NULL                NULL                      ...                             0
7990            6   1               Student             1                         ...                             1
7991          106   1               Student             1                         ...                             1

name   SUM(votes.posts.Score)   SUM(votes.posts.ViewCount)   SKEW(posts.STD(votes.BountyAmount))
role                numerical                    numerical                       unused_float
   0                        0                            0                               nan 
   1                     1384                        47460                               nan 
   2                        4                            0                               nan 
   3                    15050                       288258                               nan 
   4                     3109                        10941                               nan 
                          ...                          ...                                ...
7987                        0                            0                               nan 
7988                        4                           80                               nan 
7989                        0                            0                               nan 
7990                        1                           13                               nan 
7991                        1                            5                               nan 

name   STD(posts.SKEW(votes.BountyAmount))
role                       unused_float
   0                               nan 
   1                               nan 
   2                               nan 
   3                               nan 
   4                               nan 
                                    ...
7987                               nan 
7988                               nan 
7989                               nan 
7990                               nan 
7991                               nan 


7992 rows x 173 columns
memory usage: 10.26 MB
type: getml.DataFrame

We train an untuned XGBoostRegressor on top of featuretools' features, just like we have done for getML's features.

Since some of featuretools features are categorical, we allow the pipeline to include these features as well. Other features contain NaN values, which is why we also apply getML's Imputation preprocessor.

In [38]:
imputation = getml.preprocessors.Imputation()

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

pipe3 = getml.Pipeline(
    tags=['featuretools'],
    preprocessors=[imputation],
    predictors=[predictor],
    include_categorical=True,
)

pipe3

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=True,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=['Imputation'],
         share_selected_features=0.5,
         tags=['featuretools'])

In [39]:
pipe3.fit(featuretools_train)

Checking data model...

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Checking... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


OK.

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  XGBoost: Training as predictor... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:08


Trained pipeline.

Time taken: 0:00:08.607237.



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=True,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=['Imputation'],
         share_selected_features=0.5,
         tags=['featuretools'])

In [40]:
featuretools_score = pipe3.score(featuretools_test)
featuretools_score

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


date time             set used             target            mae       rmse   rsquared
0   2024-09-12 13:00:56   featuretools_train   Reputation    32.3818    48.2617     0.9969
1   2024-09-12 13:00:57   featuretools_test    Reputation    34.219     78.0558     0.9684

#### 2.6 Features

The most important features look as follows:

In [41]:
pipe1.features.to_sql()[pipe1.features.sort(by="importances")[0].name]

```sql
DROP TABLE IF EXISTS "FEATURE_1_87";

CREATE TABLE "FEATURE_1_87" AS
SELECT SUM( t2."score" ) AS "feature_1_87",
       t1.rowid AS rownum
FROM "USERS__STAGING_TABLE_1" t1
INNER JOIN "POSTS__STAGING_TABLE_3" t2
ON t1."id" = t2."owneruserid"
WHERE t2."posttypeid" = '2'
GROUP BY t1.rowid;
```

In [42]:
pipe2.features.to_sql()[pipe2.features.sort(by="importances")[0].name]

```sql
DROP TABLE IF EXISTS "FEATURE_1_1";

CREATE TABLE "FEATURE_1_1" AS
SELECT SUM( 
    CASE
        WHEN ( f_1_2."feature_1_2_13" > 44.522517 ) AND ( f_1_2."feature_1_2_12" > 1114.704894 ) THEN 883.3731803463131
        WHEN ( f_1_2."feature_1_2_13" > 44.522517 ) AND ( f_1_2."feature_1_2_12" <= 1114.704894 ) AND ( t2."t3__answercount" > 1.000000 ) THEN 180.6530491037182
        WHEN ( f_1_2."feature_1_2_13" > 44.522517 ) AND ( f_1_2."feature_1_2_12" <= 1114.704894 ) AND ( t2."t3__answercount" <= 1.000000 OR t2."t3__answercount" IS NULL ) THEN 117.0240683994715
        WHEN ( f_1_2."feature_1_2_13" <= 44.522517 ) AND ( f_1_2."feature_1_2_16" > 6.908843 ) AND ( t1."upvotes" > 2258.000000 ) THEN 12.25445123010827
        WHEN ( f_1_2."feature_1_2_13" <= 44.522517 ) AND ( f_1_2."feature_1_2_16" > 6.908843 ) AND ( t1."upvotes" <= 2258.000000 OR t1."upvotes" IS NULL ) THEN 14.16101412804239
        WHEN ( f_1_2."feature_1_2_13" <= 44.522517 ) AND ( f_1_2."feature_1_2_16" <= 6.908843 ) AND ( t1."upvotes__mapping_target_1_avg" > 90.840760 ) THEN 6.041694975904914
        WHEN ( f_1_2."feature_1_2_13" <= 44.522517 ) AND ( f_1_2."feature_1_2_16" <= 6.908843 ) AND ( t1."upvotes__mapping_target_1_avg" <= 90.840760 OR t1."upvotes__mapping_target_1_avg" IS NULL ) THEN -1.015010571805766
        ELSE NULL
    END
) AS "feature_1_1",
       t1.rowid AS rownum
FROM "USERS__STAGING_TABLE_1" t1
INNER JOIN "POSTS__STAGING_TABLE_3" t2
ON t1."id" = t2."owneruserid"
LEFT JOIN "FEATURES_1_2" f_1_2
ON t2.rowid = f_1_2."rownum"
GROUP BY t1.rowid;
```

#### 2.7 Productionization

It is possible to productionize the pipeline by transpiling the features into production-ready SQL code. Please also refer to getML's `sqlite3` and `spark` modules.

In [ ]:
# Creates a folder named stats_pipeline containing
# the SQL code.
pipe2.features.to_sql().save("stats_pipeline")

In [ ]:
pipe2.features.to_sql(dialect=getml.pipeline.dialect.spark_sql).save("stats_spark")

#### 2.8 Discussion

For a more convenient overview, we summarize our results into a table.

In [45]:
scores = [fastprop_score, relboost_score, featuretools_score]
pd.DataFrame(data={
    'Name': ['getML: FastProp', 'getML: Relboost', 'featuretools'],
    'R-squared': [f'{score.rsquared:.2%}' for score in scores],
    'RMSE': [f'{score.rmse:,.2f}' for score in scores],
    'MAE': [f'{score.mae:,.2f}' for score in scores]
})

Name R-squared   RMSE    MAE
0  getML: FastProp    97.77%  65.33  33.61
1  getML: Relboost    97.28%  75.66  32.27
2     featuretools    96.84%  78.06  34.22

In [ ]:
getml.engine.shutdown()

### 3. Conclusion

In this notebook, we have benchmarked several relational learning algorithms based on their ability to reverse-engineer StatsExchange's system of calculating users' reputation. We have found that all relational learning algorithm can reverse-engineer the system with high levels of accuracy, but we have also found that getML nonetheless outperforms featuretools. 

## References

Motl, Jan, and Oliver Schulte. "The CTU prague relational learning repository." arXiv preprint arXiv:1511.03086 (2015).